# Experiment 002: Grid-Based Placement

Implementing the grid-based placement approach from zaburo kernel:
- Alternating rows with 0° and 180° rotation
- Row spacing: 0.7 horizontal, 1.0 vertical
- Odd rows offset by 0.35
- Strict overlap validation

In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import time

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_side_length(trees):
    """Calculate the side length of the bounding square for a list of trees."""
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def has_overlap_strict(trees):
    """Check if any two trees overlap using STRICT criteria.
    Returns (has_overlap, details) where details contains info about overlapping pairs.
    """
    if len(trees) <= 1:
        return False, []
    
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    overlap_details = []
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx <= i:  # Skip self and already checked pairs
                continue
            
            # Check intersection
            if poly.intersects(polygons[idx]):
                intersection = poly.intersection(polygons[idx])
                area = intersection.area / (float(scale_factor) ** 2)
                
                # STRICT check: any intersection area > 1e-15 is an overlap
                if area > 1e-15:
                    overlap_details.append((i, idx, area))
    
    return len(overlap_details) > 0, overlap_details

print("Helper functions defined")

Helper functions defined


In [4]:
def find_best_trees(n):
    """Find the best grid-based placement for n trees.
    
    The approach:
    - Place trees in alternating rows (0° and 180° rotation)
    - Even rows: trees at y = row_num * 1.0, angle = 0°
    - Odd rows: trees at y = 0.8 + (row_num-1) * 1.0, angle = 180°, offset by 0.35
    - Try different numbers of trees per row to find optimal configuration
    """
    best_score = float('inf')
    best_trees = None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
                
            all_trees = []
            rest = n
            r = 0
            
            while rest > 0:
                # Determine how many trees in this row
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                # Row parameters
                angle = 0 if r % 2 == 0 else 180
                x_offset = Decimal('0') if r % 2 == 0 else Decimal('0.35')  # Half of 0.7
                
                # Y position: even rows at 0, 1.0, 2.0, ...; odd rows at 0.8, 1.8, 2.8, ...
                if r % 2 == 0:
                    y = Decimal(str(r // 2)) * Decimal('1.0')
                else:
                    y = Decimal('0.8') + Decimal(str((r - 1) // 2)) * Decimal('1.0')
                
                # Create trees for this row
                row_trees = [
                    ChristmasTree(
                        center_x=Decimal('0.7') * Decimal(str(i)) + x_offset,
                        center_y=y,
                        angle=angle
                    )
                    for i in range(m)
                ]
                all_trees.extend(row_trees)
                r += 1
            
            if len(all_trees) != n:
                continue
            
            # Calculate score (side^2)
            side = get_side_length(all_trees)
            score = side ** 2
            
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

print("Grid placement function defined")

Grid placement function defined


In [5]:
# Test with a few values
for test_n in [5, 10, 50, 100, 200]:
    score, trees = find_best_trees(test_n)
    side = math.sqrt(score)
    print(f"N={test_n:3d}: side={side:.6f}, score={score:.6f}")

N=  5: side=2.000000, score=4.000000
N= 10: side=2.200000, score=4.840000


N= 50: side=4.900000, score=24.010000


N=100: side=6.300000, score=39.690000


N=200: side=9.000000, score=81.000000


In [6]:
# Generate solutions for all N from 1 to 200
start_time = time.time()

solutions = {}  # n -> (score, trees)

for n in range(1, 201):
    score, trees = find_best_trees(n)
    solutions[n] = (score, trees)
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        side = math.sqrt(score)
        print(f"N={n:3d}: side={side:.6f}, elapsed={elapsed:.1f}s")

total_time = time.time() - start_time
print(f"\nTotal time: {total_time:.1f}s")

N= 20: side=3.150000, elapsed=0.5s


N= 40: side=4.200000, elapsed=4.1s


N= 60: side=5.200000, elapsed=13.6s


N= 80: side=5.950000, elapsed=32.4s


N=100: side=6.300000, elapsed=63.5s


N=120: side=7.000000, elapsed=109.5s


N=140: side=7.350000, elapsed=173.9s


N=160: side=8.050000, elapsed=259.9s


N=180: side=8.400000, elapsed=368.6s


N=200: side=9.000000, elapsed=505.8s

Total time: 505.8s


In [7]:
# Calculate total score
total_score = sum(score / n for n, (score, _) in solutions.items())
print(f"Total score: {total_score:.6f}")
print(f"Target score: 68.947559")
print(f"Gap: {total_score - 68.947559:.6f}")

Total score: 88.329998
Target score: 68.947559
Gap: 19.382439


In [8]:
# STRICT validation - check ALL groups for overlaps
print("Validating all configurations with STRICT overlap check...")
overlap_issues = []

for n in range(1, 201):
    _, trees = solutions[n]
    has_overlap, details = has_overlap_strict(trees)
    if has_overlap:
        overlap_issues.append((n, details))
        print(f"  N={n}: OVERLAP! {details[:3]}...")  # Show first 3 overlaps

if overlap_issues:
    print(f"\nWARNING: Overlaps found in {len(overlap_issues)} groups!")
else:
    print("\n✓ All configurations valid - no overlaps detected!")

Validating all configurations with STRICT overlap check...



✓ All configurations valid - no overlaps detected!


In [9]:
# Create submission dataframe
def to_str(x):
    return f"s{round(float(x), 6)}"

rows = []
for n in range(1, 201):
    _, trees = solutions[n]
    assert len(trees) == n, f"Expected {n} trees, got {len(trees)}"
    for i_t, tree in enumerate(trees):
        rows.append({
            "id": f"{n:03d}_{i_t}",
            "x": to_str(tree.center_x),
            "y": to_str(tree.center_y),
            "deg": to_str(tree.angle),
        })

submission = pd.DataFrame(rows)
print(f"Submission shape: {submission.shape}")
print(submission.head(20))

Submission shape: (20100, 4)
       id      x     y     deg
0   001_0   s0.0  s0.0    s0.0
1   002_0   s0.0  s0.0    s0.0
2   002_1  s0.35  s0.8  s180.0
3   003_0   s0.0  s0.0    s0.0
4   003_1   s0.7  s0.0    s0.0
5   003_2  s0.35  s0.8  s180.0
6   004_0   s0.0  s0.0    s0.0
7   004_1   s0.7  s0.0    s0.0
8   004_2  s0.35  s0.8  s180.0
9   004_3  s1.05  s0.8  s180.0
10  005_0   s0.0  s0.0    s0.0
11  005_1   s0.7  s0.0    s0.0
12  005_2  s0.35  s0.8  s180.0
13  005_3  s1.05  s0.8  s180.0
14  005_4   s0.0  s1.0    s0.0
15  006_0   s0.0  s0.0    s0.0
16  006_1   s0.7  s0.0    s0.0
17  006_2  s0.35  s0.8  s180.0
18  006_3  s1.05  s0.8  s180.0
19  006_4   s0.0  s1.0    s0.0


In [10]:
# Save submission
submission.to_csv('/home/submission/submission.csv', index=False)
submission.to_csv('/home/code/experiments/002_grid_placement/submission.csv', index=False)
print("Submission saved!")
print(f"\nFinal score: {total_score:.6f}")

Submission saved!

Final score: 88.329998
